In [ ]:
from enum import IntEnum
from glob import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from timer import timer

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from bokeh.io import output_notebook, export_png
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, NumeralTickFormatter, HoverTool, Span #, LinearAxis, Range1d
from bokeh.plotting import figure, show, output_file, save

pd.options.display.max_rows = 100
output_notebook()

In [ ]:
import jupyter_helpers
importlib.reload(jupyter_helpers)
from jupyter_helpers import bokeh_helpers, preprocess_data
importlib.reload(bokeh_helpers)
importlib.reload(preprocess_data)

# Globals
d = '/home/kapil/Desktop/polkadot-trading'
pair_addresses = set(pd.read_csv(f'{d}/data/stellaswap_liquid_pairs.csv').pair_address)
x = pd.read_csv(f'{d}/data/stellaswap_metadata_snapshot.csv')
x = x[x.pair_address.isin(pair_addresses)]
pair_d = dict(zip(x.pair_address, x.symbol))
token_d = {**dict(zip(x.token0_address, x.token0_symbol)), **dict(zip(x.token1_address, x.token1_symbol))}
pair_to_tokens = dict(zip(x.pair_address, tuple(zip(x.token0_address, x.token1_address))))
print('Token pairs:', list(pair_d.values()))
print('Tokens:', list(token_d.values()))
bh = bokeh_helpers.BokehHelper(pair_d, token_d)

def highlight(s):
    m = {
        DataRowType.SWAP_TXN: 'background-color: yellow',
        DataRowType.ON_UPDATE_TOKEN_PAIR_SNAPSHOT: 'background-color: #90ee90',
        DataRowType.END_OF_BLOCK_TOKEN_PAIR_SNAPSHOT: 'background-color: lime',
        DataRowType.ON_UPDATE_TOKEN_SNAPSHOT: 'background-color: #89cff0',
        DataRowType.END_OF_BLOCK_TOKEN_SNAPSHOT: 'background-color: #00bfff',
    }
    return [m[s.row_type] for _ in range(len(s))]

class DataRowType(IntEnum):
    END_OF_BLOCK_TOKEN_PAIR_SNAPSHOT = 1
    ON_UPDATE_TOKEN_PAIR_SNAPSHOT = 2
    END_OF_BLOCK_TOKEN_SNAPSHOT = 3
    ON_UPDATE_TOKEN_SNAPSHOT = 4
    SWAP_TXN = 5

In [ ]:
usdc_ta = '0x818ec0A7Fe18Ff94269904fCED6AE3DaE6d6dC0b'
eth_ta = '0xfA9343C3897324496A05fC75abeD6bAC29f8A40f'
bnb_ta = '0xc9BAA8cfdDe8E328787E29b4B078abf2DaDc2055'
wglmr_ta = '0xAcc15dC74880C9944775448304B263D191c6077F'
dot_ta = '0xFfFFfFff1FcaCBd218EDc0EbA20Fc2308C778080'

usdc_eth_pa = '0x0Aa48bF937ee8F41f1a52D225EF5A6F6961e39FA'
usdc_bnb_pa = '0xAc2657ba28768FE5F09052f07A9B7ea867A4608f'
wglmr_dot_pa = '0xa927E1e1E044CA1D9fe1854585003477331fE2Af'

blocks = '18[1]*'
stellaswap_files = sorted(glob(f'{d}/data/stellaswap_txn_history/all/stellaswap_data_{blocks}.feather'))
cex_files_d = {
    # 'bnb':   sorted(glob(f'{d}/data/binance_history/bnb_usdt/processed/binance_data_{blocks}')),
    # 'eth':   sorted(glob(f'{d}/data/binance_history/eth_usdt/processed/binance_data_{blocks}')),
    'bdot':   sorted(glob(f'{d}/data/binance_history/dot_usdt/processed/binance_data_{blocks}')),
    'bdot2':  sorted(glob(f'{d}/data/binance_history/dot_busd/processed/binance_data_{blocks}')),
    'bglmr':  sorted(glob(f'{d}/data/binance_history/glmr_usdt/processed/binance_data_{blocks}')),
    'bglmr2': sorted(glob(f'{d}/data/binance_history/glmr_busd/processed/binance_data_{blocks}')),
    'odot': sorted(glob(f'{d}/data/okx_history/dot_usdt/processed/okx_data_{blocks}')),
    'oglmr': sorted(glob(f'{d}/data/okx_history/glmr_usdt/processed/okx_data_{blocks}')),
}
def summarize_files(files):
    return files[:2] + ['...'] + files[-2:] if len(files) > 4 else files
summarize_files(stellaswap_files), [summarize_files(f) for f in cex_files_d.values()]

In [ ]:
%%time

df = pd.concat([pd.read_feather(f) for f in stellaswap_files]).reset_index(drop=True)
df['rate'] = df.reserve1 / df.reserve0
df['revrate'] = df.reserve0 / df.reserve1
df['block_timestamp'] = df['block_timestamp'] * 1000 # bokeh interprets epoch time in milliseconds

df = preprocess_data.compute_deltas_token(df)
df = df.groupby(['row_type', 'token_address'], dropna=False).apply(preprocess_data.add_exp_smooth_token)

df = preprocess_data.compute_deltas_token_pair(df)
df = df.groupby(['row_type', 'pair_address'], dropna=False).apply(preprocess_data.add_exp_smooth_token_pair)

# df = preprocess_data.augment_swap_rows(df, pair_to_tokens)

print("Memory usage:", df.memory_usage(index=True).sum() / 1e6, 'MB')
stella_df = df
stella_df

In [ ]:
%%time
cex_dfs = {name: pd.concat([pd.read_feather(f) for f in files]).reset_index(drop=True) for name, files in cex_files_d.items()}

for cex_df in cex_dfs.values():
    smooth_factor = 0.5
    p = cex_df['price']
    smoothed_price = [p.iloc[0]]
    for i in range(1, len(cex_df)):
        smoothed_price.append(p.iloc[i] * smooth_factor + smoothed_price[-1] * (1 - smooth_factor))
    cex_df['smoothed_price'] = smoothed_price

for name, cex_df in cex_dfs.items():
    print(f'Memory usage ({name}):', cex_df.memory_usage(index=True).sum() / 1e6, 'MB')

In [ ]:
%%time
sources = cex_dfs.keys()
cex_merged = None
for s in sources:
    df = cex_dfs[s][['timestamp', 'price', 'smoothed_price']].rename(columns={'price': f'price_{s}', 'smoothed_price': f'smoothed_price_{s}'})
    cex_merged = pd.merge_asof(cex_merged, df, on='timestamp', direction='nearest') if cex_merged is not None else df

cex_merged['max_smoothed_price_dot'] = cex_merged[['smoothed_price_bdot', 'smoothed_price_bdot2', 'smoothed_price_odot']].max(axis=1)
cex_merged['min_smoothed_price_dot'] = cex_merged[['smoothed_price_bdot', 'smoothed_price_bdot2', 'smoothed_price_odot']].min(axis=1)
cex_merged['max_smoothed_price_glmr'] = cex_merged[['smoothed_price_bglmr', 'smoothed_price_bglmr2', 'smoothed_price_oglmr']].max(axis=1)
cex_merged['min_smoothed_price_glmr'] = cex_merged[['smoothed_price_bglmr', 'smoothed_price_bglmr2', 'smoothed_price_oglmr']].min(axis=1)

pri_cols = ['timestamp', 'min_smoothed_price_dot', 'max_smoothed_price_dot', 'min_smoothed_price_glmr', 'max_smoothed_price_glmr']
cols = pri_cols + [col for col in cex_merged.columns if col not in pri_cols]
cex_merged = cex_merged[cols]
cex_merged

In [ ]:
dot_prices = stella_df[(stella_df.row_type == DataRowType.END_OF_BLOCK_TOKEN_SNAPSHOT) & (stella_df.token_address == dot_ta)][['block_timestamp', 'dai-multi_equiv_no_fees']].rename(
    columns={'dai-multi_equiv_no_fees': 'dot_stella_price'})
glmr_prices = stella_df[(stella_df.row_type == DataRowType.END_OF_BLOCK_TOKEN_SNAPSHOT) & (stella_df.token_address == wglmr_ta)][['block_timestamp', 'dai-multi_equiv_no_fees']].rename(
    columns={'dai-multi_equiv_no_fees': 'wglmr_stella_price'})
stellaswap_prices = pd.merge(dot_prices, glmr_prices, on='block_timestamp', how='inner', validate='one_to_one')

merged = pd.merge_asof(cex_merged, stellaswap_prices, left_on='timestamp', right_on='block_timestamp')
dot_mul = np.mean((merged['min_smoothed_price_dot'] + merged['max_smoothed_price_dot']) / (2e-8 * merged['dot_stella_price']))
glmr_mul = np.mean((merged['min_smoothed_price_glmr'] + merged['max_smoothed_price_glmr']) / (2 * merged['wglmr_stella_price']))
print('Stella DOT multiplier:', dot_mul)
print('Stella GLMR multiplier:', glmr_mul)

merged['adj_dot_stella_price'] = merged['dot_stella_price'] * dot_mul
merged['adj_wglmr_stella_price'] = merged['wglmr_stella_price'] * glmr_mul

merged['glmr_dot1'] = 1e3 * merged['max_smoothed_price_glmr'] / merged['min_smoothed_price_dot']
merged['glmr_dot2'] = 1e3 * merged['min_smoothed_price_glmr'] / merged['max_smoothed_price_dot']
merged['glmr_dot3'] = 1e3 * merged['adj_wglmr_stella_price'] / merged['min_smoothed_price_dot']
merged['glmr_dot4'] = 1e3 * merged['adj_wglmr_stella_price'] / merged['max_smoothed_price_dot']
merged['glmr_dot5'] = 1e11 * merged['min_smoothed_price_glmr'] / merged['adj_dot_stella_price']
merged['glmr_dot6'] = 1e11 * merged['max_smoothed_price_glmr'] / merged['adj_dot_stella_price']
merged['glmr_dot7'] = 1e11 * merged['wglmr_stella_price'] / merged['dot_stella_price']

pri_cols = ['timestamp', 'glmr_dot1', 'glmr_dot2', 'glmr_dot3', 'glmr_dot4', 'glmr_dot5', 'glmr_dot6', 'glmr_dot7']
cols = pri_cols + [col for col in merged.columns if col not in pri_cols]
merged[cols]

In [ ]:
pa = wglmr_dot_pa
df = stella_df
filters = (df.row_type == DataRowType.END_OF_BLOCK_TOKEN_PAIR_SNAPSHOT) & (df.pair_address == pa)
x = df[filters].reset_index(drop=True).dropna(axis=1, how='all')
cols = ['timestamp', 'smoothed_price_dot', 'smoothed_price_glmr', 'block_number', 'block_timestamp', 'rate']
intermediate = pd.merge_asof(cex_dfs['bdot'], cex_dfs['bglmr'], on='timestamp', direction='nearest', suffixes=('_dot', '_glmr'))
merged2 = pd.merge_asof(intermediate, x, left_on='timestamp', right_on='block_timestamp')[cols]
merged2['binance_rate'] = 1e3 * merged2['smoothed_price_glmr'] / merged2['smoothed_price_dot']
merged2['rate'] *= 1e11
mispricings = merged2[abs(merged2.binance_rate - merged2.rate) / merged2.rate > 85e-4]
vlines = [Span(location=t, dimension='height', line_color='purple', line_dash='dashed', line_width=2, line_alpha=0.25) for t in mispricings['block_timestamp'].unique()]
print(len(vlines))
merged2

In [ ]:
%%time

pa = wglmr_dot_pa
filters = (stella_df.row_type == DataRowType.END_OF_BLOCK_TOKEN_PAIR_SNAPSHOT) & (stella_df.pair_address == pa)
dd = stella_df[filters].reset_index(drop=True).dropna(axis=1, how='all')

pair = bh.plot_token_pair(dd, rate_multiplier=1e11)

wglmr_dot = pair.children[0]
wglmr_dot.width = 1250
wglmr_dot.height = 800

for label, color in zip([f'glmr_dot{i}' for i in [1,3,5]], ['red', 'green', 'brown']): #, 'purple', 'black', 'yellow', 'darkgoldenrod']):
    wglmr_dot.line('timestamp', label, source=ColumnDataSource(merged), legend_label=label, color=color, line_width=2, line_alpha=0.7)
for vline in vlines:
    wglmr_dot.add_layout(vline)

show(wglmr_dot)

In [ ]:
%%time
ta, sym, name, rm = dot_ta, 'bdot', 'DOT/USDT', 1e-8
filters = (stella_df.row_type == DataRowType.END_OF_BLOCK_TOKEN_SNAPSHOT) & (stella_df.token_address == ta)
x = bh.plot_token(stella_df[filters].reset_index(drop=True), jump_bps_thresh=100, rate_multiplier=rm)
x.line('timestamp', 'price', source=ColumnDataSource(cex_dfs[sym]), legend_label=f"Binance {name} price", color='red', line_width=1, line_alpha=0.6)
x.line('timestamp', 'smoothed_price', source=ColumnDataSource(cex_dfs[sym]), legend_label=f"Binance {name} smoothed price", color='green', line_width=1, line_alpha=0.6)
show(x)

In [ ]:
%%time
ta, sym, name, rm = wglmr_ta, 'bglmr', 'GLMR/USDT', 1
filters = (stella_df.row_type == DataRowType.END_OF_BLOCK_TOKEN_SNAPSHOT) & (stella_df.token_address == ta)
x = bh.plot_token(stella_df[filters].reset_index(drop=True), jump_bps_thresh=100, rate_multiplier=rm)
x.line('timestamp', 'price', source=ColumnDataSource(cex_dfs[sym]), legend_label=f"Binance {name} price", color='red', line_width=1, line_alpha=0.6)
x.line('timestamp', 'smoothed_price', source=ColumnDataSource(cex_dfs[sym]), legend_label=f"Binance {name} smoothed price", color='green', line_width=1, line_alpha=0.6)
show(x)